The paper addresses the challenge of singularities arising in the mean-curvature flow (MCF) of surfaces.
this papers' tries to alleviate the problem of singularities in mean-curvature flow. MCF can lead to numerical instabilities, especially due to division by zero in its discretization, causing the evolution to become singular.
the papers’ main contributions :
1) Identify that the standard discretization of MCF can result in numerical instabilities, particularly division by zero, leading to singularities during the surface evolution.
2) Introducing a modified version of MCF that eliminates the division by zero issue in the discretization. This adjustment not only stabilizes the numerical implementation but also simplifies both the discrete and continuous formulations of the flow.
3) Empirical evidence demonstrates that this modified flow provides a stable evolution for genus-zero surfaces and converges to a conformal parameterization onto the sphere.

The authors tackled this problem by proposing a reformulation of the MCF that avoids singular behavior by modifying the discretization to eliminate divisions by small or zero values, showing that this change leads to a non-singular evolution.

"find an implementation example that uses the process described in the paper."
Apply it to two different meshes and plot the results.

the process described in the paper is the Conformalized Mean Curvature Flow - example:

In [30]:
import igl
import numpy as np
import meshplot as mp
from scipy.sparse.linalg import factorized
import time


In [31]:
# Function to evolve the mesh
def make_evolutions(path, steps=30, delta=5e-4):
    vertices, faces = igl.read_triangle_mesh(path)
    updated_vertex = [vertices]

    normals = (igl.per_vertex_normals(vertices, faces) + 1) / 2
    color_values = np.linalg.norm(normals, axis=1)
    updated_colors = [color_values]

    laplacian = igl.cotmatrix(vertices, faces)
    mass_mat = igl.massmatrix(vertices, faces, igl.MASSMATRIX_TYPE_BARYCENTRIC)
    system_mat = mass_mat - delta * laplacian
    solve = factorized(system_mat)

    while steps > 0:
        vertices = solve(mass_mat @ vertices)
        normals = (igl.per_vertex_normals(vertices, faces) + 1) / 2
        color_values = np.linalg.norm(normals, axis=1)
        updated_vertex.append(vertices)
        updated_colors.append(color_values)
        steps -= 1

    return updated_vertex, faces, updated_colors

In [33]:
def plot_evolution(vertices_list, faces, colors_list=None, step=5):
    n = len(vertices_list)
    num_plots = (n + step - 1) // step
    p = None
    for i in range(0, n, step):
        idx = i // step
        v = vertices_list[i]
        c = colors_list[i] if colors_list else None
        p = mp.subplot(v, faces, s=[1, num_plots, idx], data=p, c=c, shading={"wireframe": False})



In [ ]:
verts, faces, colors = make_evolutions("mesh_data/bunny.off")
verts2, faces2, colors2 = make_evolutions("mesh_data/lion-00.off")

plot_evolution(verts, faces, colors)
plot_evolution(verts2, faces2, colors2)